In [ ]:
from datasets import load_dataset

ds = load_dataset("bigcode/the-stack-dedup",data_dir="data/java" ,streaming=True, split="train")
for sample in iter(ds):
    print(sample["content"])
    break
